### here we will standardize the data for better merge performance
process stages:
1. change country names in results table to fit noaa_countries data
2. add noaa country code from noaa_countries table to results table
3. change alpha_2 codes in results table to fit alpha_2 code in locations table
4. change city names in results table to fit city name in locations table
5. merge noaa location id to results table by city and alpha_2 code

In [1]:
import requests
import sqlite3
import pandas as pd

# NOAA API token
api_token = 'rDZYpxCxcNrIUzeserZMcYDaaHCNPcFZ'

# connect to the DB
conn = sqlite3.connect('C:/Users/Mark Rozenberg/Foobal-Climate/Data/Main_DB.db')

#### stage 1

In [48]:
# Define the mapping dictionary
country_mapping = {
    'German DR': 'Germany',
    'Czechoslovakia': 'Czech Republic',
    'Yugoslavia': 'Serbia',
    'Serbia and Montenegro': 'Serbia',
    'Scotland' : 'United Kingdom',
    'England': 'United Kingdom',
    'Wales': 'United Kingdom',
    'China PR': 'China',
    'Northern Cyprus': 'Cyprus',
    'Northern Ireland': 'Ireland',
    'Republic of Ireland': 'Ireland',
    'Puerto Rico': 'Puerto Rico [United States]',
    'Gambia': 'Gambia, The',
    'South Korea': 'Korea, South',
    'North Korea': 'Korea, North',
    'Mali Federation': 'Mali',
    'Vietnam DR': 'Vietnam',
    'Vietnam Republic': 'Vietnam',
    'Ivory Coast': 'Cote D\'Ivoire',
    'Taiwan*': 'Taiwan',
    'North Macedonia': 'Macedonia',
    'Bahamas': 'Bahamas, The',
    'Micronesia': 'Federated States of Micronesia',
    'Samoa': 'American Samoa [United States]',
    'South Sudan': 'Sudan',
    'Congo': 'Congo (Brazzaville)',
    'DR Congo': 'Congo (Kinshasa)',
    'Belgian Congo': 'Congo (Kinshasa)',
    'Bermuda': 'Bermuda [United Kingdom]',
    'Guam': 'Guam [United States]',
    'Cayman Islands': 'Cayman Islands [United Kingdom]',
    'Gibraltar': 'Gibraltar [United Kingdom]',
    'United States Virgin Islands': 'Virgin Islands [United States]',
    'Cook Islands': 'Cook Islands [New Zealand]',
    'Northern Mariana Islands': 'Northern Mariana Islands [United States]' 
}

# Update the 'results' table in the SQLite database with the new city names
for old, new in country_mapping.items():
    conn.execute('''
    UPDATE results
    SET country = ?
    WHERE country = ?
    ''', (new, old))
conn.commit()

#### stage 2

In [51]:
conn.execute('ALTER TABLE results drop column noaa_country_code')
conn.execute('ALTER TABLE results ADD COLUMN noaa_country_code TEXT')
conn.execute('''
UPDATE results
SET noaa_country_code = (
    SELECT code
    FROM noaa_countries
    WHERE results.country = noaa_countries.name
)
''')
conn.commit()

In [5]:
# check the new column
susu = pd.read_sql('SELECT distinct country, noaa_country_code FROM results', conn)
susu.to_excel('C:/Users/Mark Rozenberg/Downloads/susu.xlsx', index=False)

In [47]:
pd.read_sql('''
SELECT *
FROM noaa_countries
where name like '%tah%'
''', conn)

,code,name


#### stage 3

In [35]:
pd.read_sql_query('''
SELECT distinct city
FROM results
where city like '%ando%'
''', conn)

,city
0,San Fernando
1,Orlando
2,Landover
3,Owando
4,Andorra la Vella


In [ ]:
pd.read_sql_query('''
SELECT *
FROM locations
-- where city like '%adelaid%'
where alpha_2 in('AU','AT','AS')
''', conn)

In [ ]:
pd.read_sql_query('''
SELECT *
from countries
-- where name like '%austr%'
where alpha_2 in('AU','AT','AS')
''', conn)

In [ ]:
wuwu = pd.read_sql_query('''
with part1 as (
select distinct city, alpha_2
from results
)
SELECT r.city, l.city, r.alpha_2, l.alpha_2, l.id, l.name
FROM part1 as r
inner join locations as l
ON SUBSTR(r.city, 1, 5) = SUBSTR(l.name, 1, 5)
order by r.city
''', conn)
wuwu.to_excel('C:/Users/Mark Rozenberg/Downloads/wuwu.xlsx', index=False)

In [ ]:
# Define the mapping dictionary
city_mapping = {
    'Tel Aviv': 'Tel Aviv-Yafo',
    'Middlesbrough': 'Middlesborough'
}

# Update the 'results' table in the SQLite database with the new city names
for old_city, new_city in city_mapping.items():
    conn.execute('''
    UPDATE results
    SET city = ?
    WHERE city = ?
    ''', (new_city, old_city))
conn.commit()

In [ ]:
# Define the mapping dictionary
alpha_2_mapping = {
    'ES': 'SP',
    'IL': 'IS',
    'TR': 'TU',
    'GB': 'UK'
}

# Update the 'results' table in the SQLite database with the new city names
for old, new in alpha_2_mapping.items():
    conn.execute('''
    UPDATE results
    SET alpha_2 = ?
    WHERE alpha_2 = ?
    ''', (new, old))
conn.commit()